In [ ]:
import pandas as pd

# Load class descriptions
label_map = pd.read_csv('/content/drive/MyDrive/data_yolo/class-descriptions-boxable.csv', names=["LabelName", "DisplayName"])

# Keep only these classes
your_classes = ['Glasses', 'Picture frame', 'Remote control', 'Chair','Microwave oven','Television','Toothbrush']
label_map = label_map[label_map['DisplayName'].isin(your_classes)]

# Load bbox CSV
bbox_df = pd.read_csv('/content/drive/MyDrive/data_yolo/oidv6-train-annotations-bbox.csv')

# Filter annotations
bbox_filtered = bbox_df[bbox_df['LabelName'].isin(label_map['LabelName'])]


In [ ]:
import os
import cv2

output_txt_dir = 'labels_yolo_format'
os.makedirs(output_txt_dir, exist_ok=True)

label_dict = {row.LabelName: idx for idx, row in label_map.reset_index().iterrows()}  # e.g., /m/06ntj → 0

images_dir = '/content/drive/MyDrive/data_yolo/images'
image_filenames = os.listdir(images_dir)

for image_name in image_filenames:
    image_id = image_name.split('.')[0]
    image_path = os.path.join(images_dir, image_name)

    img = cv2.imread(image_path)
    if img is None:
        continue
    h, w = img.shape[:2]

    anns = bbox_filtered[bbox_filtered['ImageID'] == image_id]
    lines = []

    for _, row in anns.iterrows():
        class_id = label_dict[row['LabelName']]
        xmin = row['XMin'] * w
        xmax = row['XMax'] * w
        ymin = row['YMin'] * h
        ymax = row['YMax'] * h

        # Convert to YOLO format
        x_center = ((xmin + xmax) / 2) / w
        y_center = ((ymin + ymax) / 2) / h
        box_width = (xmax - xmin) / w
        box_height = (ymax - ymin) / h

        lines.append(f"{class_id} {x_center} {y_center} {box_width} {box_height}")

    # Save txt file
    with open(os.path.join(output_txt_dir, f"{image_id}.txt"), 'w') as f:
        f.write('\n'.join(lines))


In [ ]:

from sklearn.model_selection import train_test_split
import shutil

# Set base paths
label_dir = '/content/drive/MyDrive/data_yolo/labels'
image_dir = '/content/drive/MyDrive/data_yolo/images'

# Gather all label files
label_files = [f for f in os.listdir(label_dir) if f.endswith('.txt')]
label_paths = [os.path.join(label_dir, f) for f in label_files]

# Split into train and val (80% train, 20% val)
train_labels, val_labels = train_test_split(label_paths, test_size=0.2, random_state=42)

def move_files(label_list, split):
    label_dest = f'/content/labels/{split}'
    image_dest = f'/content/images/{split}'
    os.makedirs(label_dest, exist_ok=True)
    os.makedirs(image_dest, exist_ok=True)

    for label_path in label_list:
        label_name = os.path.basename(label_path)
        image_name = label_name.replace('.txt', '.jpg')

        # Copy label
        shutil.copy(label_path, os.path.join(label_dest, label_name))

        # Copy image from original image folder
        shutil.copy(os.path.join(image_dir, image_name), os.path.join(image_dest, image_name))

move_files(train_labels, 'train')
move_files(val_labels, 'val')


In [ ]:
import yaml
"""
# Assuming label_dict is already defined like: {"/m/01g317": 0, ...}
id_to_name = dict(zip(label_map.LabelName, label_map.DisplayName))
class_names = [id_to_name[k] for k, _ in sorted(label_dict.items(), key=lambda x: x[1])]

data_yaml = {
    'train': 'images/train',
    'val': 'images/val',
    'nc': len(class_names),
    'names': class_names
}

with open('data.yaml', 'w') as f:
    yaml.dump(data_yaml, f)
"""

#create the yaml file in this step, the above code logic is a little incorrect

'\n# Assuming label_dict is already defined like: {"/m/01g317": 0, ...}\nid_to_name = dict(zip(label_map.LabelName, label_map.DisplayName))\nclass_names = [id_to_name[k] for k, _ in sorted(label_dict.items(), key=lambda x: x[1])]\n\ndata_yaml = {\n    \'train\': \'images/train\',\n    \'val\': \'images/val\',\n    \'nc\': len(class_names),\n    \'names\': class_names\n}\n\nwith open(\'data.yaml\', \'w\') as f:\n    yaml.dump(data_yaml, f)\n'

In [ ]:
!git clone https://github.com/ultralytics/yolov5
%cd yolov5
!pip install -r requirements.txt



Cloning into 'yolov5'...
remote: Enumerating objects: 17516, done.
remote: Counting objects: 100% (20/20), done.
remote: Compressing objects: 100% (20/20), done.
remote: Total 17516 (delta 7), reused 0 (delta 0), pack-reused 17496 (from 4)
Receiving objects: 100% (17516/17516), 16.66 MiB | 19.01 MiB/s, done.
Resolving deltas: 100% (11998/11998), done.
/content/yolov5
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 70.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 37.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 48.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 3.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 15.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 16.1 MB/s eta

In [ ]:
%cd /content/yolov5

!python train.py \
  --img 640 \
  --batch 16 \
  --epochs 100 \
  --data /content/yolo_dataset.yaml \
  --weights yolov5s.pt \
  --name yolov5_custom

Streaming output truncated to the last 5000 lines.
      56/99      4.74G    0.03105    0.01851   0.001911         67        640:  69% 38/55 [00:23<00:11,  1.46it/s]/content/yolov5/train.py:414: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with torch.cuda.amp.autocast(amp):
      56/99      4.74G    0.03102    0.01856   0.001899         79        640:  71% 39/55 [00:24<00:12,  1.24it/s]/content/yolov5/train.py:414: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with torch.cuda.amp.autocast(amp):
      56/99      4.74G    0.03102    0.01837   0.001889         44        640:  73% 40/55 [00:24<00:09,  1.52it/s]/content/yolov5/train.py:414: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with torch.cuda.amp.autocast(amp):
      56/99      4.74G    0.03096    